In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  
os.environ["CUDA_VISIBLE_DEVICES"]="7" 

In [2]:
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn

from configs.CC import Config
from peleenet import build_net

from utils.core import anchors, init_net, get_dataloader,\
print_info, image_forward, nms_process,\
set_optimizer, set_criterion, set_logger, save_checkpoint, adjust_learning_rate,\
print_train_log, write_logger


from layers.functions import PriorBox

from tqdm import tqdm
import pickle

import torch.utils.data as data
from data import detection_collate

import time

logger = set_logger(False)

## Build net

In [3]:
cfgfile = "./configs/Pelee_VOC.py"

cfg = Config.fromfile(cfgfile)
net = build_net('train', cfg.model.input_size, cfg.model)

path2pretrained = "/workspace/raid/data/jgusak/Pelee.Pytorch/Pelee_VOC_compressed.pth"
init_net(net, cfg, path2pretrained)

ngpu = 1
if ngpu > 1:
    net = torch.nn.DataParallel(net)
if cfg.train_cfg.cuda:
    net.cuda()
    cudnn.benckmark = True


Loading resume network...


## Set optimizer, criterion

In [4]:
optimizer = set_optimizer(net, cfg)
criterion = set_criterion(cfg)

## Generate priorboxes

In [5]:

anchor_config = anchors(cfg.model)
priorbox = PriorBox(anchor_config)

priorbox = PriorBox(anchor_config)
with torch.no_grad():
    priors = priorbox.forward()
    if cfg.test_cfg.cuda:
        priors = priors.cuda()
        
num_classes = cfg.model.num_classes

## Create trainset


In [6]:
dataset_name = 'VOC'

_set = 'train_sets' # or '_test_sets'
dataset = get_dataloader(cfg, dataset_name, _set)

len(dataset)

16551

## Train

In [7]:
epoch_size = len(dataset) // (cfg.train_cfg.per_batch_size * ngpu)
max_iter = cfg.train_cfg.step_lr[-1] + 1

stepvalues = cfg.train_cfg.step_lr

print_info('===> Training STDN on ' + dataset_name, ['yellow', 'bold'])

resume_epoch=0
start_iter = resume_epoch * epoch_size if resume_epoch > 0 else 0
step_index = 0

for step in stepvalues:
    if start_iter > step:
        step_index += 1
        
        
epoch = resume_epoch        
for iteration in range(start_iter, max_iter):
        if iteration % epoch_size == 0:
            batch_iterator = iter(data.DataLoader(dataset,
                                                  cfg.train_cfg.per_batch_size * ngpu,
                                                  shuffle=True,
                                                  num_workers=cfg.train_cfg.num_workers,
                                                  collate_fn=detection_collate))
            if epoch % cfg.model.save_epochs == 0:
                save_checkpoint(net, cfg, final=False,
                                datasetname=dataset_name, epoch=epoch)
            epoch += 1
        load_t0 = time.time()
        if iteration in stepvalues:
            step_index += 1
        lr = adjust_learning_rate(
            optimizer, step_index, cfg, dataset_name)
        images, targets = next(batch_iterator)
        if cfg.train_cfg.cuda:
            images = images.cuda()
            targets = [anno.cuda() for anno in targets]
        out = net(images)
        optimizer.zero_grad()
        loss_l, loss_c = criterion(out, priors, targets)
        loss = loss_l + loss_c
        write_logger({'loc_loss': loss_l.item(),
                      'conf_loss': loss_c.item(),
                      'loss': loss.item()}, logger, iteration, status=False)
        loss.backward()
        optimizer.step()
        load_t1 = time.time()
        print_train_log(iteration, cfg.train_cfg.print_epochs,
                        [time.ctime(), epoch, iteration % epoch_size, epoch_size, iteration, loss_l.item(), loss_c.item(), load_t1 - load_t0, lr])
        
        save_checkpoint(net, cfg, final=True,
                    datasetname=dataset_name, epoch=-1)


===> Training STDN on VOC
Time:Wed Jul 31 11:45:25 2019||Epoch:1||EpochIter:0/258||Iter:0||Loss_L:0.8413||Loss_C:1.7549||Batch_Time:1.9180||LR:0.0050000
Time:Wed Jul 31 11:45:30 2019||Epoch:1||EpochIter:10/258||Iter:10||Loss_L:1.1356||Loss_C:2.5194||Batch_Time:0.2541||LR:0.0050000
Time:Wed Jul 31 11:45:34 2019||Epoch:1||EpochIter:20/258||Iter:20||Loss_L:1.2054||Loss_C:2.4542||Batch_Time:0.2573||LR:0.0050000
Time:Wed Jul 31 11:45:38 2019||Epoch:1||EpochIter:30/258||Iter:30||Loss_L:1.1262||Loss_C:2.6990||Batch_Time:0.4845||LR:0.0050000
Time:Wed Jul 31 11:45:42 2019||Epoch:1||EpochIter:40/258||Iter:40||Loss_L:1.1311||Loss_C:2.3167||Batch_Time:0.3442||LR:0.0050000
Time:Wed Jul 31 11:45:46 2019||Epoch:1||EpochIter:50/258||Iter:50||Loss_L:1.1013||Loss_C:2.3807||Batch_Time:0.2528||LR:0.0050000
Time:Wed Jul 31 11:45:50 2019||Epoch:1||EpochIter:60/258||Iter:60||Loss_L:1.0793||Loss_C:2.4759||Batch_Time:0.2535||LR:0.0050000
Time:Wed Jul 31 11:45:54 2019||Epoch:1||EpochIter:70/258||Iter:70||Loss_L

Time:Wed Jul 31 11:49:26 2019||Epoch:3||EpochIter:74/258||Iter:590||Loss_L:1.1412||Loss_C:2.3353||Batch_Time:0.2581||LR:0.0050000
Time:Wed Jul 31 11:49:30 2019||Epoch:3||EpochIter:84/258||Iter:600||Loss_L:0.9562||Loss_C:2.0343||Batch_Time:0.2576||LR:0.0050000
Time:Wed Jul 31 11:49:34 2019||Epoch:3||EpochIter:94/258||Iter:610||Loss_L:1.1387||Loss_C:2.2152||Batch_Time:0.3386||LR:0.0050000
Time:Wed Jul 31 11:49:38 2019||Epoch:3||EpochIter:104/258||Iter:620||Loss_L:1.1101||Loss_C:2.3514||Batch_Time:0.4967||LR:0.0050000
Time:Wed Jul 31 11:49:42 2019||Epoch:3||EpochIter:114/258||Iter:630||Loss_L:1.1916||Loss_C:2.1934||Batch_Time:0.2550||LR:0.0050000
Time:Wed Jul 31 11:49:46 2019||Epoch:3||EpochIter:124/258||Iter:640||Loss_L:0.9939||Loss_C:2.2305||Batch_Time:0.2550||LR:0.0050000
Time:Wed Jul 31 11:49:50 2019||Epoch:3||EpochIter:134/258||Iter:650||Loss_L:0.9535||Loss_C:2.0255||Batch_Time:0.3332||LR:0.0050000
Time:Wed Jul 31 11:49:54 2019||Epoch:3||EpochIter:144/258||Iter:660||Loss_L:0.9803||Lo

Time:Wed Jul 31 11:53:26 2019||Epoch:5||EpochIter:148/258||Iter:1180||Loss_L:0.8588||Loss_C:2.0144||Batch_Time:0.4157||LR:0.0050000
Time:Wed Jul 31 11:53:30 2019||Epoch:5||EpochIter:158/258||Iter:1190||Loss_L:0.9618||Loss_C:2.1624||Batch_Time:0.2565||LR:0.0050000
Time:Wed Jul 31 11:53:34 2019||Epoch:5||EpochIter:168/258||Iter:1200||Loss_L:1.1389||Loss_C:2.4701||Batch_Time:0.3430||LR:0.0050000
Time:Wed Jul 31 11:53:38 2019||Epoch:5||EpochIter:178/258||Iter:1210||Loss_L:1.2455||Loss_C:2.2247||Batch_Time:0.4962||LR:0.0050000
Time:Wed Jul 31 11:53:42 2019||Epoch:5||EpochIter:188/258||Iter:1220||Loss_L:1.1487||Loss_C:2.3628||Batch_Time:0.4252||LR:0.0050000
Time:Wed Jul 31 11:53:46 2019||Epoch:5||EpochIter:198/258||Iter:1230||Loss_L:1.2667||Loss_C:2.2444||Batch_Time:0.2587||LR:0.0050000
Time:Wed Jul 31 11:53:50 2019||Epoch:5||EpochIter:208/258||Iter:1240||Loss_L:1.0543||Loss_C:2.4449||Batch_Time:0.3400||LR:0.0050000
Time:Wed Jul 31 11:53:54 2019||Epoch:5||EpochIter:218/258||Iter:1250||Loss_L

Time:Wed Jul 31 11:57:25 2019||Epoch:7||EpochIter:222/258||Iter:1770||Loss_L:1.0229||Loss_C:2.1763||Batch_Time:0.2584||LR:0.0050000
Time:Wed Jul 31 11:57:29 2019||Epoch:7||EpochIter:232/258||Iter:1780||Loss_L:1.2054||Loss_C:2.3575||Batch_Time:0.5826||LR:0.0050000
Time:Wed Jul 31 11:57:33 2019||Epoch:7||EpochIter:242/258||Iter:1790||Loss_L:1.3596||Loss_C:2.1005||Batch_Time:0.2626||LR:0.0050000
Time:Wed Jul 31 11:57:36 2019||Epoch:7||EpochIter:252/258||Iter:1800||Loss_L:1.0931||Loss_C:2.2357||Batch_Time:0.3142||LR:0.0050000
Time:Wed Jul 31 11:57:42 2019||Epoch:8||EpochIter:4/258||Iter:1810||Loss_L:1.1564||Loss_C:2.0062||Batch_Time:0.2876||LR:0.0050000
Time:Wed Jul 31 11:57:45 2019||Epoch:8||EpochIter:14/258||Iter:1820||Loss_L:1.2154||Loss_C:2.0476||Batch_Time:0.2624||LR:0.0050000
Time:Wed Jul 31 11:57:49 2019||Epoch:8||EpochIter:24/258||Iter:1830||Loss_L:1.2573||Loss_C:2.0536||Batch_Time:0.4199||LR:0.0050000
Time:Wed Jul 31 11:57:54 2019||Epoch:8||EpochIter:34/258||Iter:1840||Loss_L:0.95

Time:Wed Jul 31 12:01:24 2019||Epoch:10||EpochIter:38/258||Iter:2360||Loss_L:0.9071||Loss_C:2.2690||Batch_Time:0.2588||LR:0.0050000
Time:Wed Jul 31 12:01:29 2019||Epoch:10||EpochIter:48/258||Iter:2370||Loss_L:1.0035||Loss_C:2.0589||Batch_Time:0.5129||LR:0.0050000
Time:Wed Jul 31 12:01:33 2019||Epoch:10||EpochIter:58/258||Iter:2380||Loss_L:1.0639||Loss_C:2.1416||Batch_Time:0.3413||LR:0.0050000
Time:Wed Jul 31 12:01:37 2019||Epoch:10||EpochIter:68/258||Iter:2390||Loss_L:1.1311||Loss_C:2.0954||Batch_Time:0.5077||LR:0.0050000
Time:Wed Jul 31 12:01:40 2019||Epoch:10||EpochIter:78/258||Iter:2400||Loss_L:1.1290||Loss_C:2.1273||Batch_Time:0.2513||LR:0.0050000
Time:Wed Jul 31 12:01:45 2019||Epoch:10||EpochIter:88/258||Iter:2410||Loss_L:1.2566||Loss_C:2.3317||Batch_Time:0.5005||LR:0.0050000
Time:Wed Jul 31 12:01:49 2019||Epoch:10||EpochIter:98/258||Iter:2420||Loss_L:1.2201||Loss_C:2.2763||Batch_Time:0.3332||LR:0.0050000
Time:Wed Jul 31 12:01:53 2019||Epoch:10||EpochIter:108/258||Iter:2430||Loss_

...

Time:Wed Jul 31 12:21:02 2019||Epoch:21||EpochIter:100/258||Iter:5260||Loss_L:0.8275||Loss_C:2.0350||Batch_Time:0.2637||LR:0.0050000
Time:Wed Jul 31 12:21:06 2019||Epoch:21||EpochIter:110/258||Iter:5270||Loss_L:1.0793||Loss_C:2.1454||Batch_Time:0.4329||LR:0.0050000
Time:Wed Jul 31 12:21:10 2019||Epoch:21||EpochIter:120/258||Iter:5280||Loss_L:1.0633||Loss_C:2.3011||Batch_Time:0.5737||LR:0.0050000
Time:Wed Jul 31 12:21:14 2019||Epoch:21||EpochIter:130/258||Iter:5290||Loss_L:1.0955||Loss_C:2.2370||Batch_Time:0.2603||LR:0.0050000
Time:Wed Jul 31 12:21:18 2019||Epoch:21||EpochIter:140/258||Iter:5300||Loss_L:1.0509||Loss_C:2.1665||Batch_Time:0.2506||LR:0.0050000
Time:Wed Jul 31 12:21:22 2019||Epoch:21||EpochIter:150/258||Iter:5310||Loss_L:1.0035||Loss_C:2.1954||Batch_Time:0.4218||LR:0.0050000
Time:Wed Jul 31 12:21:26 2019||Epoch:21||EpochIter:160/258||Iter:5320||Loss_L:0.8237||Loss_C:1.9717||Batch_Time:0.5864||LR:0.0050000
Time:Wed Jul 31 12:21:30 2019||Epoch:21||EpochIter:170/258||Iter:5330

Time:Wed Jul 31 12:24:57 2019||Epoch:23||EpochIter:164/258||Iter:5840||Loss_L:1.1661||Loss_C:2.4691||Batch_Time:0.2557||LR:0.0050000
Time:Wed Jul 31 12:25:01 2019||Epoch:23||EpochIter:174/258||Iter:5850||Loss_L:1.3200||Loss_C:2.2756||Batch_Time:0.2571||LR:0.0050000
Time:Wed Jul 31 12:25:05 2019||Epoch:23||EpochIter:184/258||Iter:5860||Loss_L:1.0544||Loss_C:2.0846||Batch_Time:0.5898||LR:0.0050000
Time:Wed Jul 31 12:25:09 2019||Epoch:23||EpochIter:194/258||Iter:5870||Loss_L:0.9333||Loss_C:2.0490||Batch_Time:0.3370||LR:0.0050000
Time:Wed Jul 31 12:25:13 2019||Epoch:23||EpochIter:204/258||Iter:5880||Loss_L:0.9592||Loss_C:2.1638||Batch_Time:0.2626||LR:0.0050000
Time:Wed Jul 31 12:25:17 2019||Epoch:23||EpochIter:214/258||Iter:5890||Loss_L:0.9138||Loss_C:1.9238||Batch_Time:0.2617||LR:0.0050000
Time:Wed Jul 31 12:25:21 2019||Epoch:23||EpochIter:224/258||Iter:5900||Loss_L:0.8114||Loss_C:1.9349||Batch_Time:0.6041||LR:0.0050000
Time:Wed Jul 31 12:25:25 2019||Epoch:23||EpochIter:234/258||Iter:5910

Time:Wed Jul 31 12:28:52 2019||Epoch:25||EpochIter:228/258||Iter:6420||Loss_L:1.1851||Loss_C:2.2528||Batch_Time:0.5778||LR:0.0050000
Time:Wed Jul 31 12:28:56 2019||Epoch:25||EpochIter:238/258||Iter:6430||Loss_L:1.0367||Loss_C:2.0362||Batch_Time:0.2584||LR:0.0050000
Time:Wed Jul 31 12:29:00 2019||Epoch:25||EpochIter:248/258||Iter:6440||Loss_L:1.1726||Loss_C:2.3621||Batch_Time:0.4609||LR:0.0050000
Time:Wed Jul 31 12:29:06 2019||Epoch:26||EpochIter:0/258||Iter:6450||Loss_L:1.0470||Loss_C:2.3951||Batch_Time:2.1487||LR:0.0050000
Time:Wed Jul 31 12:29:10 2019||Epoch:26||EpochIter:10/258||Iter:6460||Loss_L:1.1732||Loss_C:2.2851||Batch_Time:0.2582||LR:0.0050000
Time:Wed Jul 31 12:29:13 2019||Epoch:26||EpochIter:20/258||Iter:6470||Loss_L:1.1098||Loss_C:2.1778||Batch_Time:0.2548||LR:0.0050000
Time:Wed Jul 31 12:29:17 2019||Epoch:26||EpochIter:30/258||Iter:6480||Loss_L:1.2363||Loss_C:2.3412||Batch_Time:0.3680||LR:0.0050000
Time:Wed Jul 31 12:29:22 2019||Epoch:26||EpochIter:40/258||Iter:6490||Loss

Time:Wed Jul 31 12:32:48 2019||Epoch:28||EpochIter:34/258||Iter:7000||Loss_L:1.1612||Loss_C:2.0854||Batch_Time:0.2564||LR:0.0050000
Time:Wed Jul 31 12:32:52 2019||Epoch:28||EpochIter:44/258||Iter:7010||Loss_L:0.9844||Loss_C:2.1291||Batch_Time:0.3076||LR:0.0050000
Time:Wed Jul 31 12:32:56 2019||Epoch:28||EpochIter:54/258||Iter:7020||Loss_L:1.1832||Loss_C:2.3696||Batch_Time:0.2567||LR:0.0050000
Time:Wed Jul 31 12:33:00 2019||Epoch:28||EpochIter:64/258||Iter:7030||Loss_L:0.8907||Loss_C:2.0901||Batch_Time:0.4531||LR:0.0050000
Time:Wed Jul 31 12:33:05 2019||Epoch:28||EpochIter:74/258||Iter:7040||Loss_L:1.1433||Loss_C:2.1713||Batch_Time:0.2594||LR:0.0050000
Time:Wed Jul 31 12:33:08 2019||Epoch:28||EpochIter:84/258||Iter:7050||Loss_L:0.9634||Loss_C:2.2895||Batch_Time:0.2521||LR:0.0050000
Time:Wed Jul 31 12:33:12 2019||Epoch:28||EpochIter:94/258||Iter:7060||Loss_L:1.0641||Loss_C:2.2907||Batch_Time:0.2570||LR:0.0050000
Time:Wed Jul 31 12:33:16 2019||Epoch:28||EpochIter:104/258||Iter:7070||Loss_

KeyboardInterrupt: 

## Draft

In [23]:
from torchvision.models import resnet18, resnet50

In [26]:
mm=resnet18()

In [27]:
dict(mm.named_children()).keys()

dict_keys(['conv1', 'bn1', 'relu', 'maxpool', 'layer1', 'layer2', 'layer3', 'layer4', 'avgpool', 'fc'])

In [28]:
x= torch.randn(1, 3, 224, 224)

In [38]:
x= torch.randn(1, 3, 224, 224)

for l in net.features:
    x = l(x)
    print(l._get_name(), x.shape)
    
print('\n')
x= torch.randn(1, 3, 224, 224)
x = mm.maxpool(mm.conv1(x))

for l in [mm.layer1, mm.layer2, mm.layer3, mm.layer4]:
    x = l(x)
    print(l._get_name(), x.shape)

_StemBlock torch.Size([1, 32, 56, 56])
_DenseBlock torch.Size([1, 128, 56, 56])
conv_bn_relu torch.Size([1, 128, 56, 56])
AvgPool2d torch.Size([1, 128, 28, 28])
_DenseBlock torch.Size([1, 256, 28, 28])
conv_bn_relu torch.Size([1, 256, 28, 28])
AvgPool2d torch.Size([1, 256, 14, 14])
_DenseBlock torch.Size([1, 512, 14, 14])
conv_bn_relu torch.Size([1, 512, 14, 14])
AvgPool2d torch.Size([1, 512, 7, 7])
_DenseBlock torch.Size([1, 704, 7, 7])
conv_bn_relu torch.Size([1, 704, 7, 7])


Sequential torch.Size([1, 64, 56, 56])
Sequential torch.Size([1, 128, 28, 28])
Sequential torch.Size([1, 256, 14, 14])
Sequential torch.Size([1, 512, 7, 7])


In [41]:
net.extras

ModuleList(
  (0): conv_bn_relu(
    (conv): Conv2d(704, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (norm): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (1): conv_bn_relu(
    (conv): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (norm): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (2): conv_bn_relu(
    (conv): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (norm): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (3): conv_bn_relu(
    (conv): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (norm): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (4): conv_bn_relu(
    (conv): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (norm): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  

In [42]:
net.features

Sequential(
  (stemblock): _StemBlock(
    (stem1): conv_bn_relu(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (stem2a): conv_bn_relu(
      (conv): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (norm): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (stem2b): conv_bn_relu(
      (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (stem3): conv_bn_relu(
      (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=True)
  )
  (denseblock1): _DenseBlo